# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

import azureml.core
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset


from sklearn.model_selection import train_test_split

import pandas as pd
from azureml.widgets import RunDetails

import joblib 
import azureml.automl

from azureml.core.model import Model

from azureml.pipeline.core import PipelineData, TrainingOutput

from azureml.pipeline.core import Pipeline

import pickle

from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

from azureml.core.webservice import Webservice, AciWebservice

import logging
import os

import requests
import json

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'experiment-1'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
experiment-1,quick-starts-ws-135766,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
amlcompute_cluster_name = "capstone"

# Verify that cluster does not exist already

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [4]:
# test to see if dataset is in store
key = 'heartfailuredataset'
if key in ws.datasets.keys(): 
    dataset = ws.datasets[key] 
    print('Found dataset')

# if not, load the dataset, save it to the store
else:
    url = 'https://raw.githubusercontent.com/Priyanka2503/capstone-project/main/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(workspace=ws, name=key)

Found dataset


In [5]:
# split into data and test sets
ds = dataset.to_pandas_dataframe()
# use train test split
train, test = train_test_split(ds, random_state=625, shuffle=True)

# then write the train data to a file and load to datastore
train.to_csv('train.csv', index=False)
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

# then read to tabular dataset for automl
train = Dataset.Tabular.from_delimited_files(path = [(datastore,'train.csv')])

Uploading an estimated of 1 files
Target already exists. Skipping upload for train.csv
Uploaded 0 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
automl_settings = {
    "experiment_timeout_minutes":30,
    "enable_early_stopping":True,    
    "primary_metric":'accuracy',
    "n_cross_validations":5,    
}

automl_config = AutoMLConfig(
    compute_target=compute_target, 
    task='classification', 
    training_data=train,
    label_column_name='DEATH_EVENT',
    **automl_settings)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
# TODO: Submit your experiment
run = experiment.submit(automl_config)
RunDetails(run).show()

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [8]:
status = run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_run, model = run.get_output()

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


In [10]:
print(model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               max_depth=3,
                                                                                               min_child_weight=1,
                                                                                               missing=nan,
                                                             

In [11]:
best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":1}',
 'ensembled_iterations': '[21, 4, 9, 8, 23, 0, 1]',
 'ensembled_algorithms': "['RandomForest', 'RandomForest', 'LightGBM', 'XGBoostClassifier', 'LightGBM', 'LightGBM', 'XGBoostClassifier']",
 'ensemble_weights': '[0.125, 0.125, 0.125, 0.125, 0.25, 0.125, 0.125]',
 'best_individual_pipeline_score': '0.8618181818181819',
 'best_individual_iteration': '21',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_b3923bf1-8a73-4fe2-a388-ca7b8c4c8b62_ModelExplain',
 'model_explanation': 'True'}

In [12]:
best_run.get_metrics(name='accuracy')

{'accuracy': 0.8793939393939395}

In [13]:
#registering the model
# need to add properties, do later.

model = run.register_model(model_name ='model')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [14]:
# get the list of environments
Environment.list(workspace=ws).keys()

dict_keys(['AzureML-VowpalWabbit-8.8.0', 'AzureML-AutoML', 'AzureML-Hyperdrive-ForecastDNN', 'AzureML-PyTorch-1.4-CPU', 'AzureML-PyTorch-1.4-GPU', 'AzureML-AutoML-DNN', 'AzureML-AutoML-DNN-GPU', 'AzureML-Designer-Transform', 'AzureML-Designer', 'AzureML-AutoML-GPU', 'AzureML-Designer-IO', 'AzureML-Dask-CPU', 'AzureML-Designer-R', 'AzureML-Dask-GPU', 'AzureML-Designer-Recommender', 'AzureML-PyTorch-1.5-CPU', 'AzureML-PyTorch-1.5-GPU', 'AzureML-Sidecar', 'AzureML-Designer-CV-Transform', 'AzureML-Designer-Score', 'AzureML-Designer-PyTorch', 'AzureML-Designer-CV', 'AzureML-TensorFlow-2.1-GPU', 'AzureML-TensorFlow-2.1-CPU', 'AzureML-Designer-PyTorch-Train', 'AzureML-AutoML-DNN-Vision-GPU', 'AzureML-Designer-VowpalWabbit', 'AzureML-TensorFlow-2.2-GPU', 'AzureML-TensorFlow-2.2-CPU', 'AzureML-PyTorch-1.6-CPU', 'AzureML-PyTorch-1.6-GPU', 'AzureML-Triton', 'AzureML-TensorFlow-2.3-CPU', 'AzureML-TensorFlow-2.3-GPU', 'AzureML-DeepSpeed-0.3-GPU', 'AzureML-TensorFlow-2.0-CPU', 'AzureML-Chainer-5.1.0

In [15]:
# save the environment
my_env = Environment.get(workspace=ws, name="AzureML-AutoML")
my_env.save_to_directory('env', overwrite=True)

In [16]:
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_f3daa6fd1c2ec51a8d6be9a48b2d23d3700afd420693336f6f29ee24815c82c1_d.txt',
 'azureml-logs/65_job_prep-tvmps_f3daa6fd1c2ec51a8d6be9a48b2d23d3700afd420693336f6f29ee24815c82c1_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_f3daa6fd1c2ec51a8d6be9a48b2d23d3700afd420693336f6f29ee24815c82c1_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'explanation/3d4ca5b8/classes.interpret.json',
 'explanation/3d4ca5b8/eval_data_viz.interpret.json',
 'explanation/3d4ca5b8/expected_values.interpret.json',
 'explanation/3d4ca5b8/features.interpret.json',
 'explanation/3d4ca5b8/global_names/0.interpret.json',
 'explanation/3d4ca5b8/global_rank/0.interpret.json',
 'explanation/3d4ca5b8/global_values/0.interpret.json',
 'explanation/3d4ca5b8/local_importance_values.interpret.json',
 'explanation/3d4ca5b8/per_class_names/0.interpret.json',
 'explanati

In [17]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py','score.py')

In [18]:
# create inference_config
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(entry_script="score.py", environment = my_env)

In [19]:
#set deployment_config for webservice deployment
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)

In [22]:
#deploy the model
myservices=Model.deploy(workspace=ws,
                    name="myservices",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
myservices.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [23]:
print(myservices.get_logs())

2021-01-23T11:54:28,067882578+00:00 - iot-server/run 
2021-01-23T11:54:28,090334513+00:00 - gunicorn/run 
2021-01-23T11:54:28,090444113+00:00 - rsyslog/run 
2021-01-23T11:54:28,092802017+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [24]:
url = myservices.scoring_uri

TODO: In the cell below, send a request to the web service you deployed to test it.

In [25]:
headers = {'Content-Type':'application/json'}

test_data1 = json.dumps({'data':[{
    'age':15,
    'anaemia':0,
    'creatinine_phosphokinase':482,
    'diabetes':0,
    'ejection_fraction':20,
    'high_blood_pressure':1,
    'platelets':250000,
    'serum_creatinine':2,
    'serum_sodium':110,
    'sex':1,
    'smoking':0,
    'time':4}
    ]
        })

test_data2 = json.dumps({'data':[{
    'age':70,
    'anaemia':0,
    'creatinine_phosphokinase':231,
    'diabetes':0,
    'ejection_fraction':35,
    'high_blood_pressure':0,
    'platelets':265000,
    'serum_creatinine':1,
    'serum_sodium':180,
    'sex':1,
    'smoking':0,
    'time':150}
    ]
        })


response1 = requests.post(url, data=test_data1, headers=headers)
response2 = requests.post(url, data=test_data2, headers=headers)

print("Result:",response1.text)
print("Result:",response2.text)


Result: "{\"result\": [1]}"
Result: "{\"result\": [0]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [27]:
print(response1.status_code)
print(response1.elapsed)



200
0:00:00.163815


In [28]:
print(response2.status_code)
print(response2.elapsed)

200
0:00:00.248931


In [ ]:
service.delete()